<a href="https://colab.research.google.com/github/DanielResearch/NLP-Naive-Bayes/blob/main/MN_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasifciación de textos usando Naive Bayes Multinomial

Consideremos los siguientes elementos

* Un vovabulario $V = \{w_1,w_2,\dots,w_{|V|}\}$ de longitud $|V|$
* Un conunto de $N_D$ documentos $d \in D$, $d$ representado por un vector de palabras $X_j=\{x_{1j},x_{2j},\dots,x_{|V|j}\}$. Donde $x_{ij}$ es la frecuencia de la palabra $w_i$ en el documento $j$.
*   Un conjunto de $K$ clases $C={c_1,c_2,\dots,c_K}$

Queremos clasificar $d$ en la clase que tenga la probabildiad a posteriori más alta:

$$P(c|d)=\frac{P(d|c)P(c)}{P(d)}\propto P(d|c)P(c)$$

Esto es:

$$\hat{P}(c|d) = \arg \max_{c\in C} P(d|c)P(c)$$

donde por el supuesto de independencia condicional:
 $$P(d_j|c_k) = P(X_j|c_k) $$ \\
 $$ P(X_j|c_k) = \frac{n!}{\Pi_{i=1}^{|V|}x_{ij}!}\Pi_{i=1}^{|V|}P(w_i|c_k)^{x_{i|k}}$$\\
 $$\propto\Pi_{i=1}^{|V|}P(w_i|c_k)^{x_{ij}}$$\\

con $n=\sum_{i}^{|V|} x_{ij}$

 Para agilizar los calculos:

 $$\hat{P}(c_k|d_j)=\arg \max_{c\in C} \log{P(c_k)}+\sum_{i=1 \in |V|} x_{ij}\log{P(w_i|c_k)}$$

Los estimadores de $\hat{P}(c_k)$ y $\hat{P}(w_i|c)$

$$\hat{P}(c_k)=\frac{\text{count}(D,c_k)}{N_D}$$

$\forall k =1,\dots,K$.

$$\hat{P}(w_i|c_k)=\frac{\alpha+\sum_{j=1}^{N_D} x_{ij}f_{jk}}{\alpha|V|+\sum_{v=1}^{|V|}\sum_{j}^{N_D}x_{vj}f_{vk}}$$



Predicción:

$$P(c_k|d_j)\propto \log\hat{P}(c_k)+\sum_{i=1}^{|V|}x_{i|j}\hat{P}(w_i|c_k)$$




In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer

import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

In [ ]:
class MultiNB:
  def __init__(self,alpha=1):
    self.alpha = alpha

  """
  Calculamos la probabilidad a priori P(c_k) para toda k=1,...,K. Tenemos
  K clases
  ---
  returns
  ---
  vector con P(C_k)
  """
  def _prior(self):
    # Se crea un array P de tamaño igual al número de clases
    # para almacenar las probabilidades a priori de cada clase.
    P = np.zeros((self.n_classes_))
    # Obtener las clases únicas y las frecuencias de esas clases
    _, self.dist = np.unique(self.y, return_counts = True)
    # Se itera sobre las clases únicas y se calcula la probabilidad a priori de
    # cada clase dividiendo la frecuencia de esa clase  por el número total de
    # muestras
    for i in range(self.classes_.shape[0]):
      P[i] = np.log(self.dist[i] / self.n_samples)
    print("a priori")
    print(np.exp(P))
    return P
  """"
  Calculamos la probabilidad condicional P(w_i|c_k)
  ---
  params
  ---
  w_i: int.
       característica w_i
  i: int
     índice de la característica
  k: int
     clase en C
  ---
  return
  ---
  condicional_yi : P(w_i|c_k)
  """
  def _condicional(self, w_i , c_k):
    #  frecuencia con la que se observa la característica w_i en la clase k
    Nyi = self.N_yi[c_k,w_i]
    # Representa la frecuencia total de todoas las w_i (i=1,...,|V|) en la clase c_k
    Ny = self.N_y[c_k]
    # el numerador se calcula sumando la frecuencia de w_i en la clase k
    # y el término de suvizado Laplace
    numerador = Nyi + self.alpha
    # El denominador se calcula sumando la frecuencia total de instancias en la
    # clase k y multiplicando el término de suavizado Laplace por el
    #número total de características
    denominador = Ny + (self.alpha * self.n_features)
    # Se calcula la probabilidad condicional elevando a la x_ij
    # el cociente del numerador y el denominador
    #return x_ij * np.log(numerator / denominator)
    return Nyi * np.log(numerador / denominador)

  """
  Calcula la versosimilitud usa la fucnión condicional
  log P(d_j|c_k) = P(w_1|c_k) + P(w_2|c_k) +...+ P(w_|v| |c_k)
  ---
  params
  x: array (n_features, ) una fila de los datos
  c_k: int clase en C
  ---
  """
  def _verosimil(self, index , c_k):
    # almacenará probabilidades condicionales de cada característica w_i dada la
    # calse c_k
    tmp = 0
    # se itera sobre todas las características en el conjunto V y se calcula la
    # probabilidad condicional de cada una utilizando la función _condicional
    for w_i in index:
      tmp += self._condicional(w_i,c_k)
    # Se calcula la verosimilitud condicional multiplicando todas las
    # probabilidades condicionales calculadas.
    return tmp

  """
  Se realiza lo siguiente:
    - Obtenemos las a priors de las clases con _prior, una lista con las a
    prioris de cada clase.
    - Obtnemos la verosimilitud con ls función _verosimil
  ---
  params
  ---
  X: array 2D (n_samples, n_features) Multinomial
  y: array 1D (n_samples,) labels en enteros
  """
  def fit( self, X, y ):
    # etiquetas de clase
    self.y = y
    # # de muestras y características en los datos de entrada
    self.n_samples, self.n_features = X.shape
    # Las clases únicas
    self.classes_ = np.unique( y )
    # # de clases en el conjunto de datos
    self.n_classes_ = self.classes_.shape[ 0 ]
    # Se calculan las probabilidades a priori utilizando la función _prior
    self.class_priors_ = self._prior()
    # Se inicializan las matrices para contar la frecuencia de características
    # para cada clase.
    # feature count
    self.N_yi = np.zeros( ( self.n_classes_, self.n_features ))
    #total count
    self.N_y = np.zeros( ( self.n_classes_ ) )
    # Se itera sobre las clases y se calculan las frecuencias de las
    # características para cada clase. Las frecuencias se almacenan en
    # las matrices
    print("for")
    for i in self.classes_: # eje de las xs
      print("Clase")
      print(i)
      indices = np.argwhere (self.y == i ).flatten()
      columnwise_sum = []
      for j in range( self.n_features ): # eje de las ys
        columnwise_sum.append( np.sum( X[ indices, j ] ) )
      print("matriz frecuencia w_i en clase c_k")
      print(columnwise_sum)
      print("matriz frecuencia total en clase c_k")
      print(np.sum( columnwise_sum ))
      self.N_yi[i] = columnwise_sum #2d
      self.N_y[i] = np.sum( columnwise_sum ) #1d
    print(self.N_yi)
    print(self.N_y)

  """
  Predicción del modelo
  ---
  params
  ---
  X 2d array
    (n_samples, n_features)
  ---
  """
  def predict( self, X ):
    # Obtiene el número de muestras y características de los datos de entrada
    doc, feartures = X.shape
    #  Inicializa un array para almacenar las probabilidades predichas para
    # cada clase y cada muestra.
    self.predict_proba = np.zeros( ( doc, self.n_classes_ ) )
    c_k = 0
    #  itera sobre cada documento en los datos de entrada
    for d in range( doc ):
      #Inicializa un array para almacenar la verosimilitud conjunta para cada
      #clase dada la muestra actual.
      w_i = np.argwhere(X[d,] != 0)
      for c_k in range( self.n_classes_ ):
        # P(c_k)P(w_i|c_k)
        self.predict_proba[d,c_k] = self.class_priors_[ c_k ]*self._verosimil(
            w_i, c_k)
    self.class_predict = np.argmax(self.predict_proba, axis = 1)
    return self.class_predict



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Doctorado/4to cuatrimestre/Datos/ejemplo .csv')
print(dataset.shape)
dataset.head(10)

(6, 2)


,document,category
0,just plain boring,-1
1,entirely predictable and lacks energy,-1
2,no surprises and very few laughs,-1
3,very powerful,1
4,the most fun film of the summer,1
5,predictable with no fun,-1


In [ ]:
cv = CountVectorizer()
X = cv.fit_transform(df['document']).toarray()
lb = LabelBinarizer()
y = lb.fit_transform(df['category']).ravel()
print(X.shape,y.shape)
print(df.shape)


In [ ]:
X_train = X[0:5,:]
y_train = y[0:5,]

X_test = X[5:7,:]
y_test = y[5:7,]

In [ ]:
modelo_object = MultiNB()

modelo_object.fit( X_train, y_train )
y_hat_object = modelo_object.predict( X_test )

print( accuracy_score(y_test, y_hat_object) )